In [51]:
# !pip install -r ../requirements.txt
!pip install python-dotenv

In [55]:

from dotenv import load_dotenv
from pathlib import Path
import os

curr_dir = Path(__file__).parent
env_path = curr_dir / '.env' 
load_dotenv(dotenv_path=env_path, override=True)
account_key = os.getenv("ACCOUNT_KEY")
print(account_key)

NameError: name '__file__' is not defined

In [20]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [21]:
credential

In [22]:
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [23]:
# !python ./src/diabetes-training.py

Loading Data...
Traceback (most recent call last):
  File "/mnt/batch/tasks/shared/LS_root/mounts/clusters/cic27d604846864aa694/code/Users/richardyantas/azure-dp100/lab1/./src/diabetes-training.py", line 12, in <module>
    diabetes = pd.read_csv('diabetes.csv')
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 482, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 811, in __init__
    self._engine = self._make_engine(self.engine)
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pandas/io/parsers/reader

In [24]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    # compute="my_VM2",
    display_name="diabetes-pythonv2-train",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

HttpResponseError: (UserError) Unknown compute target 'my_VM2'.
Code: UserError
Message: Unknown compute target 'my_VM2'.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "78e4631aa0880857281800d3fd40670d",
        "request": "9c68ab2c48368037"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2024-10-09T00:46:23.0209031+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerError": {
            "code": "UnknownTargetType",
            "innerError": null
        }
    }
}

In [25]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

workspaceartifactstore
workspaceworkingdirectory
workspaceblobstore
workspacefilestore


In [26]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="mlwdp100storagec89ca8ff0",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key=config["ACCOUNT_KEY"]
    ),
)

ml_client.create_or_update(store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blob_training_data', 'description': 'Blob Storage for training data', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourceGroups/rg-dp100-lc27d604846864aa694/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-lc27d604846864aa694/datastores/blob_training_data', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cic27d604846864aa694/code/Users/richardyantas/azure-dp100/lab1', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f426c61d210>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x7f4288cd3d90>, 'container_name': 'training-data', 'account_name': 'mlwdp100storagec89ca8ff0', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [27]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_training_data
workspaceartifactstore
workspaceworkingdirectory
workspaceblobstore
workspacefilestore


In [29]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './src/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data asset pointing to a local file, automatically uploaded to the default datastore",
    name="diabetes-local"
)

ml_client.data.create_or_update(my_data)

Uploading diabetes.csv (< 1 MB): 0.00B [00:00, ?B/s] (< 1 MB): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 518k

Data({'path': 'azureml://subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourcegroups/rg-dp100-lc27d604846864aa694/workspaces/mlw-dp100-lc27d604846864aa694/datastores/workspaceblobstore/paths/LocalUpload/d52d15a0d7d1e95b90a03f146099424a/diabetes.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-local', 'description': 'Data asset pointing to a local file, automatically uploaded to the default datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourceGroups/rg-dp100-lc27d604846864aa694/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-lc27d604846864aa694/data/diabetes-local/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cic27d604846864aa694/code/Users/richardyantas/azure-dp100/lab1',

In [30]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

datastore_path = 'azureml://datastores/blob_training_data/paths/data-asset-path/'

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Data asset pointing to data-asset-path folder in datastore",
    name="diabetes-datastore-path"
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourcegroups/rg-dp100-lc27d604846864aa694/workspaces/mlw-dp100-lc27d604846864aa694/datastores/blob_training_data/paths/data-asset-path/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-datastore-path', 'description': 'Data asset pointing to data-asset-path folder in datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourceGroups/rg-dp100-lc27d604846864aa694/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-lc27d604846864aa694/data/diabetes-datastore-path/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cic27d604846864aa694/code/Users/richardyantas/azure-dp100/lab1', 'creation_context': <azure.ai.ml.entities._sys

In [ ]:
upload(src_dir, target_path=None, overwrite=False, show_progress=True)

In [35]:

# ml_client.datastores.upload_files(
#     datastore_name="blob_training_data",
#     files=["./src/dataprueba.txt"],
#     target_path=f"{datastore_path}/dataprueba.txt",
#     overwrite=True  # Esto sobrescribirá archivos existentes si se establecen
# )


Method mount: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


TypeError: DatastoreOperations.mount() missing 1 required positional argument: 'path'

In [44]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = './src'

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="diabetes-table"
)

ml_client.data.create_or_update(my_data)

Uploading src (0.52 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 519409/519409 [00:00<00:00, 6930450.03it/s]


Data({'path': 'azureml://subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourcegroups/rg-dp100-lc27d604846864aa694/workspaces/mlw-dp100-lc27d604846864aa694/datastores/workspaceblobstore/paths/LocalUpload/e7208069c0826f96f95127545eeaecda/src/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['./diabetes.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-table', 'description': 'MLTable pointing to diabetes.csv in data folder', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/41463841-d87e-4f36-8ea3-7b0294e63525/resourceGroups/rg-dp100-lc27d604846864aa694/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-lc27d604846864aa694/data/diabetes-table/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cic27d604846864aa694/code/Users/richardyantas/azure-dp100/lab1', 'creation_context': <azure.ai.m

In [45]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

diabetes-local
diabetes-datastore-path
diabetes-table
